In [1]:
import pandas as pd 
import numpy  as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler , PowerTransformer 
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

## import data

In [2]:
data = pd.read_csv( 'heart.csv' ) 

### Change output col to 0-1

In [3]:
data['output'] = data['output'] - 1 
data['exercise'] = data['exercise']-1

### Selecting cols by type 

In [4]:
Y = data['output']
X = data.drop('output',axis=1)

### Split in train and test 

In [5]:
X_train,X_test,y_train,y_test = train_test_split( X,Y,random_state=72,test_size=0.15 )

In [6]:
numeric_columns = ['age','restbp','chol','maxhr','dep','fluor']
categorical_features = [ 'chestpain','ecg','thal']
others_features = set( X_train.columns ) - set( categorical_features ) - set( numeric_columns ) 

### Function to preprocessing numeric columns

In [41]:
def preprocessing_standard(data_to_standard,padronizacao):
    numeric_columns = ['age','restbp','chol','maxhr','dep','fluor']
    
    if padronizacao == 'Standard':
        scaler = StandardScaler()
        scaler.fit(X_train[numeric_columns])
        return pd.DataFrame(scaler.transform( data_to_standard[numeric_columns] ) , columns = data_to_standard[numeric_columns].columns)
    if padronizacao == 'MinMax':
        minmax = MinMaxScaler() 
        minmax.fit( X_train[numeric_columns] )
        return pd.DataFrame( minmax.transform( data_to_standard[numeric_columns] ), columns = data_to_standard[numeric_columns].columns )

    
def preprocessing_categorical(data_to_standard):
    enc = OneHotEncoder( handle_unknown='error' , sparse= False,drop='first' ).fit( X_train[categorical_features] )
    return pd.DataFrame(enc.transform( data_to_standard[categorical_features] ) , columns = enc.get_feature_names( categorical_features ) )

### Aplicando preprocessamento

In [42]:
train = pd.concat([
                 preprocessing_categorical( X_train ),
                 X_train[others_features].reset_index().drop("index",axis=1),
                 preprocessing_standard(X_train,padronizacao='MinMax')
                 ],axis=1
                 )


test = pd.concat([
                 preprocessing_categorical( X_test ),
                 X_test[others_features].reset_index().drop("index",axis=1),
                 preprocessing_standard(X_test,'MinMax')
                 ],
                 axis=1
                 )

In [43]:
from sklearn.linear_model import LogisticRegression
parametros_regularizacao = ['l1', 'l2', 'elasticnet','none']
fit_log = LogisticRegression().fit(train,y_train)